NECESSARY INSTALLS

In [1]:
# Install dependencies
%pip install supersuit
%pip install gymnasium
%pip install box2d-py
%pip install swig
%pip install "gymnasium[atari,accept-rom-license]"
%pip install "stable-baselines3[extra]"
%pip install "pettingzoo[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached AutoROM-0.4.2-py3-none-any.whl (16 kB)
  Using cached Shimmy-0.2.1-py3-none-any.whl (25 kB)
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 1.3.0
    Uninstalling Shimmy-1.3.0:
      Successfully uninstalled Shimmy-1.3.0
  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.6.1
    Uninstalling AutoROM-0.6.1:
      Successfully uninstalled AutoROM-0.6.1
Note: you may need to restart the kernel to use updated packages.
  Using cached Shimmy-1.3.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached AutoROM-0.6.1-py3-none-any.whl (9.4 kB)
Using cached Shimmy-1.3.0-py3-none-any.whl (37 kB)
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 0.2.1
    Unins

In [2]:
%pip install pettingzoo

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install supersuit

Note: you may need to restart the kernel to use updated packages.


IMPORT LIBRARIES

In [1]:
import gymnasium as gym
import supersuit as ss
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch.nn as nn
import torch
import torch.nn.functional as F

In [2]:
class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=512):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # Modify the following layers to fit your observation shape
        self.cnn = nn.Sequential(
            nn.Conv2d(observation_space.shape[0], 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Flatten()
        )
        # Compute shape by doing one forward pass
        with torch.no_grad():
            n_flatten = self.cnn(torch.as_tensor(observation_space.sample()[None]).float()).shape[1]
        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations):
        return self.linear(self.cnn(observations))

In [3]:
# Check for CUDA device
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the base environment
env = gym.make('ALE/Pong-v5')

# Apply the specified wrappers
env = ss.color_reduction_v0(env, mode='B')  # Reduces the color of frames to black and white
env = ss.resize_v1(env, x_size=84, y_size=84)  # Resize the observation space
env = ss.frame_stack_v1(env, 4)  # Stack 4 frames together
env = ss.dtype_v0(env, dtype='float32')  # Change the data type of observations
env = ss.normalize_obs_v0(env, env_min=0, env_max=1)  # Normalize the observation space

# Because the environment is now a non-vectorized environment after applying wrappers,
# we have to vectorize it using DummyVecEnv
env = DummyVecEnv([lambda: env])

# Define the custom policy kwargs with the custom CNN and device
policy_kwargs = {
    'features_extractor_class': CustomCNN,
    'features_extractor_kwargs': {'features_dim': 512},
    'normalize_images': False
}

# Instantiate the agent on the specified device
model = PPO("CnnPolicy", env, verbose=1, policy_kwargs=policy_kwargs, device=device)

# Train the agent
model.learn(total_timesteps=100000)


Using device: cpu


A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


Using cpu device


KeyboardInterrupt: 

In [ ]:
model.save("ppo_pong_right_side_v1")

In [ ]:
model = PPO.load("ppo_pong_right_side", env=env)

In [ ]:
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
    if dones:
        obs = env.reset()
